# Importing llamas_pyjamas modules

**Please follow the installation instructions prior to attempting this**

In [1]:
import os
import sys
import numpy as np
import pickle
import ray
import pkg_resources
import glob
import traceback
from   pathlib import Path
from   llamas_pyjamas.config import BASE_DIR, OUTPUT_DIR, DATA_DIR

# Get package root and add to path before other imports as a precaution -> if installed as package this should hopefully not be needed
package_root = Path().absolute().parent
sys.path.append(str(package_root))
sys.path.append(BASE_DIR+'/')


ray.init(ignore_reinit_error=True)
import llamas_pyjamas.Trace.traceLlamasMulti as trace # type: ignore
import llamas_pyjamas.Extract.extractLlamas as extract # type: ignore
from llamas_pyjamas.Image.WhiteLight import WhiteLight, WhiteLightFits

2025-04-23 00:55:57,248	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/var/folders/jp/s9gkt_zs0fncvspy757cxv0c0000gn/T/ipykernel_68395/2899916872.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
2025-04-23 00:56:01,010	INFO worker.py:1819 -- Started a local Ray instance.
2025-04-23 00:56:02,600	INFO worker.py:1652 -- Calling ray.init() again after it has already been called.


Fibre map path: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/LUT/LLAMAS_FiberMap_rev02.dat


In [2]:
# Get absolute path to llamas_pyjamas package to check the installation
package_path = pkg_resources.resource_filename('llamas_pyjamas', '')
package_root = os.path.dirname(package_path)

print(f"Package path: {package_path}")
print(f"Package root: {package_root}")

Package path: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas
Package root: /Users/simcoe/GIT/llamas-pyjamas


**Optional Ray initialisation if attempting to parallelise some of the processes**

In [3]:
# Configure Ray runtime environment
runtime_env = {
    "py_modules": [package_root],
    "env_vars": {"PYTHONPATH": f"{package_root}:{os.environ.get('PYTHONPATH', '')}"},
    "excludes": [
        str(Path(DATA_DIR) / "**"),  # Exclude DATA_DIR and all subdirectories
        "**/*.fits",                 # Exclude all FITS files anywhere
        "**/*.gz",                 # Exclude all tarballs files anywhere
        "**/*.zip",                 # Exclude all zip files anywhere
        "**/*.pkl",                  # Exclude all pickle files anywhere
        "**/.git/**",               # Exclude git directory
    ]
}

# Initialize Ray
ray.shutdown()
ray.init(runtime_env=runtime_env)

2025-04-18 00:21:53,434	INFO worker.py:1819 -- Started a local Ray instance.
2025-04-18 00:21:53,719	INFO packaging.py:600 -- Creating a file package for local module '/Users/simcoe/GIT/llamas-pyjamas'.
2025-04-18 00:21:53,869	INFO packaging.py:392 -- Pushing file package 'gcs://_ray_pkg_ed4bae3ad5d59f94.zip' (4.08MiB) to Ray cluster...
2025-04-18 00:21:53,907	INFO packaging.py:405 -- Successfully pushed file package 'gcs://_ray_pkg_ed4bae3ad5d59f94.zip'.


Python version:,3.12.7
Ray version:,2.39.0


In [3]:
##This is also optional, but it can help with debugging
DATA_DIR = "/Users/simcoe/Science/LLAMAS/CommissioningRun/NIGHT5"
DATA_DIR = "/Users/simcoe/Science/LLAMAS/RUN02/2025-03-05"
import importlib
importlib.reload(trace)

<module 'llamas_pyjamas.Trace.traceLlamasMulti' from '/Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/Trace/traceLlamasMulti.py'>

# Extraction

To run the extraction process on a single fits image and then plot a single spectrum:

In [ ]:
from llamas_pyjamas.GUI.guiExtract import GUI_extract
filepath = 'Path_to_your_fits_file'
# Example: filepath = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/comissioning_data/20241128/LLAMAS_2024-11-28T03_50_39.584_mef.fits

#This should take a few mintues to run and in your output folder you should see the extracted pickle file
GUI_extract(filepath)


## General code for picking a fibre to plot the spectrum of

In [ ]:

#Example: extract_pickle = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/llamas-pyjamas/llamas_pyjamas/output/LLAMASExtract_batch_20250203_215658.pkl'
extract_pickle = 'output/your_extraction_file.pkl'


with open(extract_pickle, 'rb') as f:
    exobj = pickle.load(f)

extraction_list = exobj['extractions']

#select which extraction object you wish to plot
# Each object represents a single HDU extension in the FITS file which was extracted. 
# The order should correspond to the order in the fits file 
# (but don't forget that the fits file has a primary HDU as well so the first detector image is HDU 1 in the fits but index 0 in the extraction list)

HDU_idx = 0 #change the '0' to the value of the HDU you wish to plot spectra from
spec_arrays = extraction_list[HDU_idx].counts #change the '0' to the value of the HDU you wish to plot spectra from

#This gives you an array of the extracted spectra for the selected HDU, shape = (Nfib, 2048)
#You can plot the spectra for a single fiber using the following code

import matplotlib.pyplot as plt
Nfib = 0 #select the fiber you wish to plot
plt.plot(spec_arrays[Nfib])
plt.show()


### To try and find which fibre the object is in:

In [ ]:
idx = [0, 3, 6, 9, 12, 15, 18, 21] ### These are the indexes which represent the red detector outputs, 1A, 1B, 2A, 2B, 3A, 3B, 4A, 4B
                                ## If you want the green detector outputs +1 to each index, for blue +2 for each index


## This code will go through each detector output and print the maximum summed value and the row with the maximum summed value
#think of this as trying to find the fibre your object is most likely to be on
for i in idx:
    data = exobj['extractions'][i].counts
    row_sums = np.nansum(data, axis=1)
    max_sum = np.nanmax(row_sums)
    row_with_max_sum = np.argmax(row_sums)
    print(i, max_sum, row_with_max_sum)

idx_with_obj = 0 #select the index of the detector output you wish to extract the spectra from    
row_with_object = 0 #select the row with the object you wish to extract based on what the loop prints out
#You should pick the the idx and row where the maximum sum is highest, this is the most likely fibre the object is on      
spectra = exobj['extractions'][idx_with_obj].counts[row_with_max_sum]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.plot(spectra)
plt.ylabel('Counts')
plt.show()

# If you want to extract a specific detector rather than the whole IFU image:

In [ ]:
from llamas_pyjamas.Extract.extractLlamas import ExtractLlamas
import pickle
from astropy.io import fits

### MAKE SURE THE COLOUR AND BENCHSIDE OF THE TRACEFILE YOU LOAD IN MATCHES THE HDU YOU WANT TO EXTRACT
#e.g. red_1A typically corresponds to hdu[1] in the fits file
#To check this print hdu[i].header
with open(DATA_DIR + '/Your_trace_file.pkl', 'rb') as f:
    traceobj = pickle.load(f)

hdu = fits.open('Path_to_your_fits_file.fits')
ex = ExtractLlamas(traceobj, hdu[hdu_index].data.astype(float), dict(hdu[hdu_index].header))

## Extract the flat field (for getting fiber throughputs later)

In [5]:
# from llamas_pyjamas.GUI.guiExtract import GUI_extract
import llamas_pyjamas.GUI.guiExtract as ge
importlib.reload(ge)

skyflat_filename = os.path.join(DATA_DIR, 'LLAMAS_2025-03-05T23_24_13.255_mef.fits')
# trace_filename = os.path.join(DATA_DIR, 'LLAMAS_2024-11-29T23_50_11.041_mef.fits')
trace_filename = os.path.join(DATA_DIR, 'LLAMAS_2025-03-05T23_24_13.255_mef.fits')
ge.GUI_extract(skyflat_filename)

file is /Users/simcoe/Science/LLAMAS/RUN02/2025-03-05/LLAMAS_2025-03-05T23_24_13.255_mef.fits


2025-04-18 00:22:11,119	INFO worker.py:1819 -- Started a local Ray instance.
2025-04-18 00:22:11,272	INFO packaging.py:600 -- Creating a file package for local module '/Users/simcoe/GIT/llamas-pyjamas'.
2025-04-18 00:22:11,418	INFO packaging.py:392 -- Pushing file package 'gcs://_ray_pkg_caf068cdd13061b6.zip' (4.08MiB) to Ray cluster...
2025-04-18 00:22:11,455	INFO packaging.py:405 -- Successfully pushed file package 'gcs://_ray_pkg_caf068cdd13061b6.zip'.


Trimmed HDU 1 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 2 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 3 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 4 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 5 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 6 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 7 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 8 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 9 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 10 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 11 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 12 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 1

(pid=2547) Calling ray.init() again after it has already been called.


(pid=2546) Fibre map path: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/LUT/LLAMAS_FiberMap_rev02.dat
(process_trace pid=2547) Optimal True
(process_trace pid=2547) bench 1 self.side A channel green
(process_trace pid=2546) bench 1 self.side A channel red
(process_trace pid=2549) bench 1 self.side B channel red
(process_trace pid=2548) bench 1 self.side A channel blue
(process_trace pid=2550) bench 1 self.side B channel green


...

(process_trace pid=2547) bench 4 self.side B channel blue


(process_trace pid=2548) 2025-04-18 00:26:03,134 - llamas_pyjamas.Extract.extractLlamas - INFO - Benchside 4A counts shape (300, 2048) [repeated 4x across cluster]
(process_trace pid=2547) 2025-04-18 00:25:26,122 - llamas_pyjamas.Extract.extractLlamas - INFO - No dead fibers entry found in LUT for bench=4, side=B [repeated 2x across cluster]
2025-04-18 00:26:17,698 - llamas_pyjamas.Extract.extractLlamas - INFO - Saving batch extraction to: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/output/LLAMAS_2025-03-05T23_24_13.255_extract.pkl


Extraction list = [<llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x17da5e510>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x20009fda0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x20c1e5a90>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b8b2e9c0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b8ffcf20>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bbe44350>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bd9e77a0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bdb7b650>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bd850d70>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b8990440>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bd1fcfe0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2be0068a0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bd1c

2025-04-18 00:26:26,097 - llamas_pyjamas.Extract.extractLlamas - INFO - Loaded 24 extractions


obj = [<llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bbe12870>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c01601d0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bf8f9ca0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2be94d490>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bfc32840>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bf92c830>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bfe86c00>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bffb1b80>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bfef4560>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c365e720>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c3cb94c0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b5f7a180>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b59e8f20>, <llam

In [6]:
importlib.reload(extract)
flat_picklename = os.path.join(OUTPUT_DIR, 'LLAMAS_2024-11-29T23_50_11.041_extract.pkl')
flatdict = extract.ExtractLlamas.loadExtraction(flat_picklename)
# dir(flatdict['extractions'][18])

2025-04-18 00:26:36,433	INFO worker.py:1652 -- Calling ray.init() again after it has already been called.


In [ ]:
flatdict['extractions'][10].relative_throughput[5] = 0.96
flatdict['extractions'][10].relative_throughput

## Extract an arc frame

In [8]:
arc_filename   = os.path.join(DATA_DIR, 'LLAMAS_2025-03-05T19_04_46.449_mef.fits')
ge.GUI_extract(arc_filename)

file is /Users/simcoe/Science/LLAMAS/RUN02/2025-03-05/LLAMAS_2025-03-05T19_04_46.449_mef.fits


2025-04-18 00:28:51,621	INFO worker.py:1819 -- Started a local Ray instance.
2025-04-18 00:28:51,772	INFO packaging.py:600 -- Creating a file package for local module '/Users/simcoe/GIT/llamas-pyjamas'.
2025-04-18 00:28:51,925	INFO packaging.py:392 -- Pushing file package 'gcs://_ray_pkg_ba16d6507a78ec77.zip' (4.02MiB) to Ray cluster...
2025-04-18 00:28:51,963	INFO packaging.py:405 -- Successfully pushed file package 'gcs://_ray_pkg_ba16d6507a78ec77.zip'.


Trimmed HDU 1 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 2 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 3 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 4 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 5 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 6 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 7 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 8 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 9 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 10 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 11 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 12 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 1

(pid=2693) Calling ray.init() again after it has already been called.


(process_trace pid=2693) Optimal True
(process_trace pid=2693) bench 1 self.side A channel green
(process_trace pid=2692) bench 1 self.side A channel red
(process_trace pid=2696) bench 1 self.side B channel green
(process_trace pid=2695) bench 1 self.side B channel red
(process_trace pid=2694) bench 1 self.side A channel blue


...

(process_trace pid=2695) bench 4 self.side B channel blue


(process_trace pid=2695) 2025-04-18 00:31:55,693 - llamas_pyjamas.Extract.extractLlamas - INFO - No dead fibers entry found in LUT for bench=4, side=B [repeated 3x across cluster]
(process_trace pid=2696) 2025-04-18 00:32:31,819 - llamas_pyjamas.Extract.extractLlamas - INFO - Benchside 4A counts shape (300, 2048) [repeated 5x across cluster]
2025-04-18 00:32:45,581 - llamas_pyjamas.Extract.extractLlamas - INFO - Saving batch extraction to: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/output/LLAMAS_2025-03-05T19_04_46.449_extract.pkl


Extraction list = [<llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c59c7f80>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c2163e30>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c0186f90>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b6fa5760>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c59c6900>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x110ddcec0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c6e39a30>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c52a39b0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c5230e60>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c59c68a0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bf6e6090>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c0163380>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b42c

2025-04-18 00:32:55,799 - llamas_pyjamas.Extract.extractLlamas - INFO - Loaded 24 extractions


obj = [<llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b3c59250>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c28271a0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2ad442870>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2bad467b0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b5819e20>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c4e45940>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2ada1c4a0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2addbfa40>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c54bc9b0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2ad645b50>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2ade86330>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2ad3c7d40>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2b26c91f0>, <llam

### Read in and plot up arc extractions

In [140]:
importlib.reload(extract)
import pickle
arc_filename   = os.path.join(DATA_DIR, 'LLAMAS_2025-03-05T19_04_46.449_mef.fits')
arc_picklename = os.path.join(OUTPUT_DIR, os.path.basename(arc_filename).replace('_mef.fits', '_extract.pkl'))
with open(arc_picklename, 'rb') as fp:
    batch_data = pickle.load(fp)
arcdict = extract.ExtractLlamas.loadExtraction(arc_picklename)
arcspec, metadata = arcdict['extractions'], arcdict['metadata']

2025-04-20 23:51:32,757	INFO worker.py:1652 -- Calling ray.init() again after it has already been called.


In [141]:
import llamas_pyjamas.Arc.arcLlamas as arc
importlib.reload(arc)
importlib.reload(extract)
arc.shiftArcX(arc_picklename)

2025-04-20 23:51:56,293	INFO worker.py:1652 -- Calling ray.init() again after it has already been called.


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
shift_picklename = arc_picklename.replace('_extract.pkl', '_extract_shifted.pkl')
#arcspec_shifted, metadata_shifted = extract.load_extractions(shift_picklename)
#[plt.plot(arcspec_shifted[18].xshift[i,:], arcspec_shifted[18].counts[i,:],".") for i in range(100,200)]
#plt.show()

In [ ]:
importlib.reload(arc)
#flatdict = extract.ExtractLlamas.loadExtraction(flat_picklename)
#flatdict['extractions'][10].relative_throughput[10] = 100
#flatdict['extractions'][10].relative_throughput
# arcdict = extract.ExtractLlamas.loadExtraction(shift_picklename)
# gd = (arcdict['extractions'][18].xshift[150] > 100)
# plt.plot(flatdict['extractions'][18].counts[150][gd])
# plt.show()

arc.fiberRelativeThroughput(flat_picklename, shift_picklename)

1A red Fiber #0:  Throughput = 1.018
1A red Fiber #1:  Throughput = 0.998
1A red Fiber #2:  Throughput = 0.965
1A red Fiber #3:  Throughput = 1.024
1A red Fiber #4:  Throughput = 0.983
1A red Fiber #5:  Throughput = 1.040
1A red Fiber #6:  Throughput = 1.039
1A red Fiber #7:  Throughput = 1.034
1A red Fiber #8:  Throughput = 0.933
1A red Fiber #9:  Throughput = 1.042
1A red Fiber #10:  Throughput = 1.000
1A red Fiber #11:  Throughput = 1.028
1A red Fiber #12:  Throughput = 1.050
1A red Fiber #13:  Throughput = 1.043
1A red Fiber #14:  Throughput = 1.047
1A red Fiber #15:  Throughput = 0.928
1A red Fiber #16:  Throughput = 1.032
1A red Fiber #17:  Throughput = 1.046
1A red Fiber #18:  Throughput = 1.007
1A red Fiber #19:  Throughput = 0.893
1A red Fiber #20:  Throughput = 0.759
1A red Fiber #21:  Throughput = 0.803
1A red Fiber #22:  Throughput = 1.028
1A red Fiber #23:  Throughput = 0.984
1A red Fiber #24:  Throughput = 0.663
1A red Fiber #25:  Throughput = 0.784
1A red Fiber #26:  Thr

2025-04-20 11:37:54,970 - llamas_pyjamas.Extract.extractLlamas - INFO - Saving batch extraction to: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/output/LLAMAS_2025-03-05T19_04_46.449_extract_shifted_shifted_tp.pkl


4A red Fiber #143:  Throughput = 1.075
4A red Fiber #144:  Throughput = 0.960
4A red Fiber #145:  Throughput = 0.899
4A red Fiber #146:  Throughput = 1.055
4A red Fiber #147:  Throughput = 0.923
4A red Fiber #148:  Throughput = 0.997
4A red Fiber #149:  Throughput = 1.074
4A red Fiber #150:  Throughput = 1.000
4A red Fiber #151:  Throughput = 1.026
4A red Fiber #152:  Throughput = 1.101
4A red Fiber #153:  Throughput = 0.998
4A red Fiber #154:  Throughput = 1.091
4A red Fiber #155:  Throughput = 1.059
4A red Fiber #156:  Throughput = 0.777
4A red Fiber #157:  Throughput = 0.923
4A red Fiber #158:  Throughput = 0.725
4A red Fiber #159:  Throughput = 0.969
4A red Fiber #160:  Throughput = 1.095
4A red Fiber #161:  Throughput = 1.092
4A red Fiber #162:  Throughput = 1.059
4A red Fiber #163:  Throughput = 1.096
4A red Fiber #164:  Throughput = 1.083
4A red Fiber #165:  Throughput = 1.044
4A red Fiber #166:  Throughput = 1.099
4A red Fiber #167:  Throughput = 0.985
4A red Fiber #168:  Throu

In [222]:
ray.shutdown()

In [ ]:
tp=shift_picklename.replace('.pkl','_shifted_tp.pkl')
importlib.reload(arc)
arc.arcSolve(tp)

Loading arc extraction
Fitting wavelength solution
Processing 4A red
Transferring arc solution to all fibers in LLAMAS red channels...
Processing 4A green
Transferring arc solution to all fibers in LLAMAS green channels...
Processing 4A blue


2025-04-20 13:19:26,640 - llamas_pyjamas.Extract.extractLlamas - INFO - Saving batch extraction to: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/output/LLAMAS_reference_arc.pkl


Transferring arc solution to all fibers in LLAMAS blue channels...
Saving wavelength solution to disk


()

In [9]:
arcdict = extract.ExtractLlamas.loadExtraction(os.path.join(OUTPUT_DIR, 'LLAMAS_reference_arc.pkl'))

Now extract a standard star spectrum

In [5]:
import llamas_pyjamas.GUI.guiExtract as ge
importlib.reload(ge)
std_filename   = os.path.join(DATA_DIR, 'LLAMAS_2025-03-06T00_15_52.794_mef.fits')
ge.GUI_extract(std_filename)

file is /Users/simcoe/Science/LLAMAS/RUN02/2025-03-05/LLAMAS_2025-03-06T00_15_52.794_mef.fits


2025-04-23 00:57:27,591	INFO worker.py:1819 -- Started a local Ray instance.
2025-04-23 00:57:27,786	INFO packaging.py:600 -- Creating a file package for local module '/Users/simcoe/GIT/llamas-pyjamas'.
2025-04-23 00:57:27,947	INFO packaging.py:392 -- Pushing file package 'gcs://_ray_pkg_32f3940613c358dd.zip' (4.52MiB) to Ray cluster...
2025-04-23 00:57:27,990	INFO packaging.py:405 -- Successfully pushed file package 'gcs://_ray_pkg_32f3940613c358dd.zip'.


Trimmed HDU 1 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 2 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 3 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 4 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 5 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 6 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 7 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 8 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 9 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 10 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 11 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 12 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
Trimmed HDU 1

(pid=68445) Calling ray.init() again after it has already been called.


(pid=68445) Fibre map path: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/LUT/LLAMAS_FiberMap_rev02.dat
(process_trace pid=68445) Bias file: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/mastercalib/combined_bias.fits
(process_trace pid=68445) Trimmed HDU 1 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048]
(pid=68452) Fibre map path: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/LUT/LLAMAS_FiberMap_rev02.dat [repeated 7x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(process_trace pid=68452) Bias file: /Users/simcoe/GIT/llamas-pyjamas/llamas_pyjamas/mastercalib/combined_bias.fits [repeated 7x across cluster]
(process_trace pid=68445) Trimmed HDU 23 from (2048, 2048) to (2048, 2048) based on DATASEC=[1:2048, 1:2048] [repeated 176x across cluster]
(process_trace pid=68445) Bias index:

...

(process_trace pid=68445) bench 4 self.side B channel green
(process_trace pid=68451) bench 4 self.side B channel blue


(process_trace pid=68452) 2025-04-23 01:00:01,376 - llamas_pyjamas.Extract.extractLlamas - INFO - No dead fibers entry found in LUT for bench=4, side=A [repeated 2x across cluster]
(process_trace pid=68449)  RuntimeWarning: invalid value encountered in scalar divide (extractLlamas.py:165)
(process_trace pid=68451) 2025-04-23 01:00:05,013 - llamas_pyjamas.Extract.extractLlamas - INFO - No dead fibers entry found in LUT for bench=4, side=B [repeated 2x across cluster]
(process_trace pid=68448) 2025-04-23 01:01:06,708 - llamas_pyjamas.Extract.extractLlamas - INFO - Benchside 3B counts shape (300, 2048)
(raylet) Spilled 3435 MiB, 41 objects, write throughput 412 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(process_trace pid=68449) 2025-04-23 01:01:12,210 - llamas_pyjamas.Extract.extractLlamas - INFO - Benchside 4B counts shape (298, 2048) [repeated 5x across cluster]
2025-04-23 01:01:26,813 - llamas_pyjamas.Extract.extractLlamas - INFO - Saving batch extraction to: /Users/

Extraction list = [<llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x1fea48680>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c1b71040>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x17c8fc0e0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c2205fd0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c42762d0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c77260c0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c8774230>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c87fc0b0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c8867290>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c893ab40>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c89da390>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c6a3f800>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c8a8

2025-04-23 01:01:37,789 - llamas_pyjamas.Extract.extractLlamas - INFO - Loaded 24 extractions


obj = [<llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c287ab70>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c9c0f6e0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2ca0599a0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2cabbadb0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c9eeede0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2ca6a0f80>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2cabf7890>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2c974bb90>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2cdbd4e30>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2ce0cffb0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2ce8c2d20>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2cef31af0>, <llamas_pyjamas.Extract.extractLlamas.ExtractLlamas object at 0x2cf5b0890>, <llam

Load the extracted standard

In [6]:
std_dict = extract.ExtractLlamas.loadExtraction(os.path.join(OUTPUT_DIR,'LLAMAS_2025-03-06T00_15_52.794_extract.pkl'))
std_extract = std_dict['extractions']

In [10]:
import llamas_pyjamas.Arc.arcLlamas as arc
importlib.reload(arc)
std_wvcal = arc.arcTransfer(std_dict, arcdict)

In [58]:
import matplotlib.pyplot as plt
%matplotlib qt
import Flux.calcThroughput as flux
importlib.reload(flux)
importlib.reload(arc)
flux.calcThroughput(std_dict, arcdict, 'blue')
flux.calcThroughput(std_dict, arcdict, 'green')
flux.calcThroughput(std_dict, arcdict, 'red')
plt.ylim(0,100)
plt.show()

[-1.04477539  3.          2.2583505  ...  0.87452827  0.34102726
  1.05573048]
[2.13693861 9.59997194 6.1242219  ... 2.42215668 3.38969975 1.80300328]
[-1.04331891 -0.31960218 -1.9989198  ...  7.56901308  4.5929114
  9.86947017]
